# 데이터 확인

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('Database/CIT_FDG_2017_2018.xlsx', engine='openpyxl')
df

## 년도별 병합

In [ ]:
import pandas as pd
import os

# 📂 파일 경로
data_dir = "Database"

# 🔤 FDG 파일 목록
fdg_files = [
    "CIT_FDG_2017_2018.xlsx",
    "CIT_FDG_2019_2020.xlsx",
    "CIT_FDG_2021_2022.xlsx",
    "CIT_FDG_2023_20240731.xlsx"
]

# 🔤 MR 파일 목록
mr_files = [
    "CIT_MR_2017_2018.xlsx",
    "CIT_MR_2019_2020.xlsx",
    "CIT_MR_2021_2022.xlsx",
    "CIT_MR_2023_20240731.xlsx"
]

def merge_and_save(xlsx_list, output_name):
    df_list = []
    for file in xlsx_list:
        file_path = os.path.join(data_dir, file)
        try:
            df = pd.read_excel(file_path)
            df_list.append(df)
            print(f"✅ 불러옴: {file}")
        except Exception as e:
            print(f"❌ 오류: {file} - {e}")

    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        merged_df.to_csv(f"{output_name}.csv", index=False, encoding="utf-8-sig")
        print(f"📁 저장 완료: {output_name}.csv")
    else:
        print(f"⚠️ 병합할 데이터가 없습니다: {output_name}")

# 🚀 실행
merge_and_save(fdg_files, "Database/CIT_FDG_2017_20240731")
merge_and_save(mr_files, "Database/CIT_MR_2017_20240731")

In [ ]:
df_CIT_FDG = pd.read_csv('Database/CIT_FDG_2017_20240731.csv', encoding='utf-8-sig')
df_CIT_FDG.groupby('(실명)등록번호').size()

In [ ]:
df_CIT_MR = pd.read_csv('Database/CIT_MR_2017_20240731.csv', encoding='utf-8-sig')
df_CIT_MR.groupby('(실명)등록번호').size()

In [ ]:
print(df_CIT_FDG.columns.tolist())

In [ ]:
print(df_CIT_MR.columns.tolist())

In [ ]:
import pandas as pd

# CSV 파일 경로
file_path = 'Database/CIT_MR_2017_20240731.csv'

# 열 이름만 읽기 (데이터는 불러오지 않음)
df_columns = pd.read_csv(file_path, encoding='utf-8-sig', nrows=0)

# 열 이름과 인덱스 출력
for idx, col in enumerate(df_columns.columns):
    if idx in [0, 68, 72]:
        print(f"{idx}: {col}")

# 전체 병합 

## 데이터 불러오기

In [ ]:
import pandas as pd

# 1. 데이터 불러오기
df_mr  = pd.read_csv('Database/CIT_MR_2017_20240731.csv', encoding='utf-8-sig')
df_fdg = pd.read_csv('Database/CIT_FDG_2017_20240731.csv', encoding='utf-8-sig')

- 최종 환자 수 검수

In [ ]:
# 1. df_mr에 존재하는 환자 등록번호 리스트 추출
mr_patient_ids = df_mr['(실명)등록번호'].unique()

# 2. df_fdg에서 FDG 기록 있는 사람 중 등록번호만 추출
fdg_patient_ids = df_fdg[df_fdg['처방 처방코드명'].str.contains('FDG', na=False)]['(실명)등록번호'].unique()

# 3. 교집합 구하기
common_ids = set(mr_patient_ids) & set(fdg_patient_ids)

# 4. 결과 출력
print(f"df_mr 환자 수: {len(mr_patient_ids)}")
print(f"df_fdg에 FDG 기록이 있는 df_mr 환자 수: {len(common_ids)}")

## 각 컬럼별로 구분

- ‘영상 처방코드명’이 CIT 영상이고 ‘처방 처방코드명’이 MRI 또는 FDG야
- ’진단 입원(외래)일자‘를 기준으로 가장 최근 날짜인 행만 남기면 돼
- 그리고 나서 최종적으로 CIT MR FDG 모두 있는 환자만 골라주면 돼

In [ ]:
# 1) 접미사 없이 남길 “고정” 진단·환자 정보
fixed_cols = [
    '(실명)등록번호','(실명)내원번호','(실명)생년월일',
    '지역병원코드','연구등록번호','연구내원번호',
    '진단 성별코드','진단 성별','진단 나이',
    '진단 내원구분코드','진단 내원구분명',
    '진단 입원(외래)일자','진단 입원(외래)과 코드','진단 입원(외래)과명',
    '진단 입원(외래)주치의ID','진단 입원(외래)주치의명',
    '진단 진단명입력일자','진단 진단입력과코드','진단 진단입력과명',
    '진단 주상병여부','진단 주상병여부(계산)','진단 확진여부',
    '진단 진단코드','진단 진단영문명','진단 진단한글명',
    '진단 ICD10Cd','진단 ICD10Cd명'
]

# 2) CIT 영상 관련 컬럼 (CIT_ 접두사)
cit_img_cols = [
    '영상 성별코드','영상 성별','영상 검사시나이',
    '영상 내원구분코드','영상 내원구분명',
    '영상 처방코드','영상 처방코드명','영상 처방일자',
    '영상 검사시행일자','영상 검사시행시간',
    '영상 처방과코드','영상 처방과명',
    '영상 처방주치의ID','영상 처방주치의명',
    '영상 검사실코드','영상 검사실명',
    '영상 조영제코드','영상 조영제명','영상 조영제투여량','영상 조영제처방비고',
    '영상 처방일련번호','영상 AccNo',
    '영상 판독일자','영상 판독의ID','영상 판독의명','영상 판독결과',
    '영상 전문의ID','영상 전문의명'
]

# 3) MRI 처방 관련 컬럼 (MRI_ 접두사)
mri_rx_cols = [
    '처방 성별코드','처방 성별','처방 처방시나이',
    '처방 내원구분코드','처방 내원구분명',
    '처방 처방코드','처방 처방코드명','처방 처방일자','처방 처방시행일자',
    '처방 처방과코드','처방 처방과명',
    '처방 주치의 ID','처방 주치의명',
    '처방 시행과코드','처방 시행과명',
    '처방 시행자 ID','처방 시행자명',
    '처방 처방명(메시지)','처방 처방구분','처방 처방구분명',
    '처방 처방분류코드','처방 처방분류코드명',
    '처방 T용량','처방 용량','처방 횟수','처방 일수',
    '처방 용법코드','처방 용법코드명',
    '처방 처방일련번호','처방 수가코드','처방 투여시간코드',
    '처방 치식정보','처방 치식식별명'
]

# 4) FDG 처방 관련 컬럼 (FDG_ 접두사)
fdg_rx_cols = mri_rx_cols.copy()  # MRI와 동일한 처방 필드 목록

In [ ]:
# --- CIT-MR 쪽 접두사 붙이기 ---
df_mr = (
    df_mr
    # 1) 고정 정보는 그대로
    # 2) CIT 영상 정보 → CIT_ 접두사
    .rename(columns={col: f'CIT_{col}'   for col in cit_img_cols})
    # 3) MRI 처방 정보 → MRI_ 접두사
    .rename(columns={col: f'MRI_{col}'   for col in mri_rx_cols})
)

# --- CIT-FDG 쪽 접두사 붙이기 ---
df_fdg = (
    df_fdg
    # 1) 고정 정보는 그대로
    # 2) CIT 영상 정보 → CIT_ 접두사
    .rename(columns={col: f'CIT_{col}'   for col in cit_img_cols})
    # 3) FDG 처방 정보 → FDG_ 접두사
    .rename(columns={col: f'FDG_{col}'   for col in fdg_rx_cols})
)

In [ ]:
print(df_mr.columns.tolist())
print(df_fdg.columns.tolist())

## 각각 최신 행 추출

In [ ]:
# 진단 입원(외래)일자 : 두 데이터 프레임 중에서 제일 최신 값으로 바꾸고 최신_진단 입원(외래)일자 컬럼 생성
# MR에서 MRI_처방시행일자이 최신인 행에서 MRI suffix 붙어있는 애들 다 가져오고 FDG에서 FDG_처방시행일자이 최신인 행에서 FDG suffix 붙어있는 애들 다 가져오기
# CIT는 MRI_처방시행일자이 최신인 행과 FDG_처방시행일자이 최신인 행에서 CIT_검사시행일자가 동일한지 확인하고 로그 출력
# 동일하면 그 해당 행에서 CIT suffix 붙어있는 애들 내용이 같은지 확인하고 로그 출력
# 동일하지 않으면 오류 로그 출력

### 각각 최신행 확인

In [ ]:
# MR과 FDG에서 각각 최신 처방 시행일자를 가진 행을 추출

mr_latest = (df_mr
             .sort_values('MRI_처방 처방시행일자')
             .groupby('(실명)등록번호', as_index=False)
             .tail(1))
fdg_latest = (df_fdg
              .sort_values('FDG_처방 처방시행일자')
              .groupby('(실명)등록번호', as_index=False)
              .tail(1))

mr_latest.shape, fdg_latest.shape


In [ ]:
### 최신 CIT 일치
# 1) 시리즈로 뽑아서 인덱스를 (실명)등록번호 로 설정
mr_dates = mr_latest.set_index('(실명)등록번호')['CIT_영상 검사시행일자']
fdg_dates = fdg_latest.set_index('(실명)등록번호')['CIT_영상 검사시행일자']

# 2) datetime으로 변환
mr_dates = pd.to_datetime(mr_dates)
fdg_dates = pd.to_datetime(fdg_dates)

# 3) 공통 환자 ID만 골라내기
common_ids = mr_dates.index.intersection(fdg_dates.index)

# 4) 같은 순서로 재인덱싱
mr_aligned = mr_dates.loc[common_ids]
fdg_aligned = fdg_dates.loc[common_ids]

# 5) 비교해서 불일치 여부 Series 생성
mismatch = mr_aligned != fdg_aligned

# 6) 일치하는 ID만 리스트로 저장
match_ids = common_ids[~mismatch].tolist()

if not mismatch.any():
    print("✅ 모든 환자에 대해 CIT_영상 검사시행일자가 일치합니다.")
else:
    bad_ids = common_ids[mismatch].tolist()
    print(f"❌ {len(bad_ids)}명 불일치:")
    for pid in bad_ids:
        mr_dt = mr_aligned.loc[pid]
        fdg_dt = fdg_aligned.loc[pid]
        # Timestamp 객체이므로 .date() 호출 가능
        print(f"  Patient {pid}: MR={mr_dt.date()}, FDG={fdg_dt.date()}")

print(f"✅ 총 {len(match_ids)}명 일치: {match_ids}")

### fixed_cols

In [ ]:
import pandas as pd

# — 가정: df_mr, df_fdg, fixed_cols, match_ids 가 정의되어 있습니다.

# 1) match_ids 대상의 fixed_cols 데이터만 뽑아서, 날짜 컬럼 변환
dfs = []
for df in (df_mr, df_fdg):
    tmp = (
        df[df['(실명)등록번호'].isin(match_ids)]
        .loc[:, fixed_cols]     # fixed_cols 안에 진단 입원(외래)일자 포함
        .copy()
    )
    tmp['진단 입원(외래)일자'] = pd.to_datetime(tmp['진단 입원(외래)일자'])
    dfs.append(tmp)

# 2) 두 프레임을 합쳐서
all_diag = pd.concat(dfs, ignore_index=True)

# 3) 환자별 가장 최신 진단일자 행만 남기기
latest_diag = (
    all_diag
    .sort_values('진단 입원(외래)일자')
    .groupby('(실명)등록번호', as_index=False)
    .tail(1)
)

# 4) fixed_cols 컬럼만 남긴 최종 결과
fixed_latest = latest_diag[fixed_cols].reset_index(drop=True)

print("▶ 결과 환자 수:", fixed_latest['(실명)등록번호'].nunique())
print("fixed_latest.columns.tolist():", fixed_latest.columns.tolist())
fixed_latest.head()

### 나머지

In [ ]:
# 3) MR에서 (실명)등록번호 + cit_img_cols + mri_rx_cols 컬럼만 추출
mr_select_cols = ['(실명)등록번호'] + \
                 [f'CIT_{col}' for col in cit_img_cols] + \
                 [f'MRI_{col}' for col in mri_rx_cols]
df_mr_select = mr_latest[mr_select_cols].copy()

# 4) FDG에서 (실명)등록번호 + fdg_rx_cols 컬럼만 추출
fdg_select_cols = ['(실명)등록번호'] + \
                  [f'FDG_{col}' for col in fdg_rx_cols]
df_fdg_select = fdg_latest[fdg_select_cols].copy()

In [ ]:
df_mr_select.head()

In [ ]:
df_fdg_select.head()

In [ ]:
# 전체 중에 최신 진단일자 기준으로 fixed_cols 받아오고
# 그 과정에서 진단 입원(외래)일자 컬럼 명을 최신 진단 입원(외래)일자 로 바꾸고
# CIT_영상 검사시행일자가 같은 환자에 대해서만 FDG 쪽의 CIT 컬럼 cit_img_cols 받아오고 나머지 MRI와 FDG 쪽의 mri_rx_cols랑 fdg_rx_cols받아와서 하나의 데이터프레임으로 합치고 싶어.

In [ ]:
# 가정: fixed_df 는 이미 “fixed_cols” 정보만 담고 있는 DataFrame입니다.
#       df_mr_select, df_fdg_select 는 위에서 만든 MR·FDG 선택 DataFrame입니다.

# 1) fixed_df 기준으로 df_mr_select 붙이기 (left join)
merged = fixed_latest.merge(
    df_mr_select,
    on='(실명)등록번호',
    how='left'
)

# 2) 여기에 다시 df_fdg_select 붙이기 (left join)
merged = merged.merge(
    df_fdg_select,
    on='(실명)등록번호',
    how='left'
)

# 3) 결과 확인
print("▶ 최종 shape:", merged.shape)

#4) 저장
merged.to_csv("Database/CIT_MR_FDG_merged.csv", index=False, encoding= 'utf-8-sig')

merged.head()

In [52]:
# 1) 남기고 싶은 컬럼 리스트
desired = [
    '(실명)등록번호',
    #— 진단 정보
    '(실명)생년월일',
    '진단 성별코드',
    '진단 나이',
    '진단 입원(외래)일자',
    '진단 입원(외래)주치의명',
    '진단 진단명입력일자',
    '진단 진단입력과명',
    '진단 확진여부',
    '진단 진단영문명',
    '진단 진단한글명',

    #— CIT 영상
    'CIT_영상 처방코드명',
    'CIT_영상 검사시행일자',
    'CIT_영상 검사시행시간',
    'CIT_영상 판독일자',
    'CIT_영상 판독의명',
    'CIT_영상 판독결과',
    
    #— MRI 처방
    'MRI_처방 처방코드명',
    'MRI_처방 처방시행일자',
    
    #— FDG 처방
    'FDG_처방 처방코드명',
    'FDG_처방 처방시행일자',
]

# 2) 실제 존재하는 컬럼만 걸러내기
keep = [c for c in desired if c in merged.columns]

# 3) 추출
df_final_merged = merged[keep].copy()

# 4) 확인
print("최종 컬럼:", df_final_merged.columns.tolist())
print("shape:", df_final_merged.shape)

# 5) 저장
df_final_merged.to_csv('Database/CIT_MR_FDG_processed.csv', index=False, encoding='utf-8-sig')
print("✅ df_final_merged.csv 저장 완료")

최종 컬럼: ['(실명)등록번호', '(실명)생년월일', '진단 성별코드', '진단 나이', '진단 입원(외래)일자', '진단 입원(외래)주치의명', '진단 진단명입력일자', '진단 진단입력과명', '진단 확진여부', '진단 진단영문명', '진단 진단한글명', 'CIT_영상 처방코드명', 'CIT_영상 검사시행일자', 'CIT_영상 검사시행시간', 'CIT_영상 판독일자', 'CIT_영상 판독의명', 'CIT_영상 판독결과', 'MRI_처방 처방코드명', 'MRI_처방 처방시행일자', 'FDG_처방 처방코드명', 'FDG_처방 처방시행일자']
shape: (574, 21)
✅ df_final_merged.csv 저장 완료
